In [1]:
%load_ext autoreload
%autoreload 2

import sys; 
sys.path.insert(0, '../../esinet')
sys.path.insert(0, '../')

import numpy as np
from copy import deepcopy
from scipy.sparse.csgraph import laplacian
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
from invert.cmaps import parula
pp = dict(surface='white', hemi='both')

In [2]:
info = get_info(kind='biosemi64')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
leadfield -= leadfield.mean(axis=0)
n_chans, n_dipoles = leadfield.shape
dist = cdist(pos, pos)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    1.5s remaining:    1.5s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.7s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished


In [63]:
# settings = dict(number_of_sources=1, extents=40, duration_of_trial=0.01, target_snr=99999999999)
settings = dict(number_of_sources=5, extents=(1, 2), duration_of_trial=0.05, target_snr=1e99)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()
y = evoked.data
x = stc.data
y -= y.mean(axis=0)

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 31.83it/s]

Using control points [0.00000000e+00 0.00000000e+00 5.44663175e-08]


For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Linearly Constrained minimum variance Beamformer

In [64]:
alpha = 0.00 * np.linalg.svd(leadfield)[1].max()
leadfield -= leadfield.mean(axis=0)
I = np.identity(n_chans)
C_inv = np.linalg.inv(y@y.T + alpha * I)
W = []
for i in range(n_dipoles):
    l = leadfield[:, i][:, np.newaxis]
    w = np.linalg.inv(l.T @ C_inv @ l ) * l.T @ C_inv
    W.append(w)
W = np.stack(W, axis=1)[0].T
W = W / np.linalg.norm(W, axis=0)
inverse_operator = W.T
x_hat = inverse_operator@y
stc_ = stc.copy()
stc_.data = x_hat / x_hat.max()
stc_.plot(**pp, brain_kwargs=dict(title="LCMV"))

Using control points [3.08976656e-08 3.74650135e-08 6.41728830e-01]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Weight-normalized minimum variance Beamformer

In [5]:
alpha = 0.0001 * np.linalg.svd(leadfield)[1].max()

I = np.identity(n_chans)
C_inv = np.linalg.inv(y@y.T + alpha * I)
C_inv_2 = np.linalg.inv(C_inv)

W = []
for i in range(n_dipoles):
    l = leadfield[:, i][:, np.newaxis]
    # w = np.linalg.inv(l.T @ C_inv @ l ) * l.T @ C_inv
    w = (C_inv @ l) / np.sqrt(l.T @ C_inv_2 @ l)
    W.append(w)
W = np.stack(W, axis=1)[:, :, 0]
# W = W / np.linalg.norm(W, axis=0)
inverse_operator = W.T
x_hat = inverse_operator@y
stc_ = stc.copy()
stc_.data = x_hat
stc_.plot(**pp, brain_kwargs=dict(title="WNMV"))

Using control points [3.06071338e-05 3.34228022e-05 4.75232311e-05]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Standardized minimum variance Beamformer

In [6]:
alpha = 0.00001 #* np.linalg.svd(leadfield)[1].max()

I = np.identity(n_chans)
C_inv = np.linalg.inv(y@y.T + alpha * I)
C_inv_2 = np.linalg.inv(C_inv)

W = []
for i in range(n_dipoles):
    l = leadfield[:, i][:, np.newaxis]
    # w = np.linalg.inv(l.T @ C_inv @ l ) * l.T @ C_inv
    w = (C_inv @ l) / np.sqrt(l.T @ C_inv @ l)
    W.append(w)
W = np.stack(W, axis=1)[:, :, 0]
# W = W / np.linalg.norm(W, axis=0)
inverse_operator = W.T
x_hat = inverse_operator@y
stc_ = stc.copy()
stc_.data = x_hat
stc_.plot(**pp, brain_kwargs=dict(title="SMV"))

Using control points [0.00824484 0.00900333 0.01281906]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Higher-Order minimum variance Beamformer

In [7]:
alpha = 0.00 * np.linalg.svd(leadfield)[1].max()
n = 3
I = np.identity(n_chans)
C_inv = np.linalg.inv(y@y.T + alpha * I)
C_inv_n = deepcopy(C_inv)

for _ in range(n-1):
    C_inv_n = np.linalg.inv(C_inv_n)

W = []
for i in range(n_dipoles):
    l = leadfield[:, i][:, np.newaxis]
    # w = np.linalg.inv(l.T @ C_inv @ l ) * l.T @ C_inv
    w = (C_inv_n @ l) / (l.T @ C_inv_n @ l)
    W.append(w)
W = np.stack(W, axis=1)[:, :, 0]
W = W / np.linalg.norm(W, axis=0)
inverse_operator = W.T
x_hat = inverse_operator@y
stc_ = stc.copy()
stc_.data = x_hat
stc_.plot(**pp, brain_kwargs=dict(title="HOC-MV"))

Using control points [3.34309532e-19 3.59768349e-19 2.83108806e-13]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Eigenspace-based minimum variance Beamformer

In [8]:
from invert.util import find_corner
alpha = 0.00 * np.linalg.svd(leadfield)[1].max()

I = np.identity(n_chans)
C = y@y.T

U, s, _ = np.linalg.svd(C)
j = find_corner(np.arange(len(s)), s)

Us = U[:, :j]
Un = U[:, j:]


C_inv = np.linalg.inv(C + alpha * I)

W = []
for i in range(n_dipoles):
    l = leadfield[:, i][:, np.newaxis]
    # w = np.linalg.inv(l.T @ C_inv @ l ) * l.T @ C_inv
    w_mv = (C_inv @ l) / (l.T @ C_inv @ l)
    w_esmv = Us @ Us.T @ w_mv
    W.append(w_esmv)

W = np.stack(W, axis=1)[:, :, 0]
# W = W / np.linalg.norm(W, axis=0)

inverse_operator = W.T
x_hat = inverse_operator@y
stc_ = stc.copy()
stc_.data = x_hat / x_hat.max()
stc_.plot(**pp, brain_kwargs=dict(title="EIGMV"))

Using control points [4.01961344e-15 7.73175377e-15 4.87323889e-01]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Multiple Constrained minimum variance Beamformer

In [9]:
L = deepcopy(leadfield)
L -= L.mean(axis=0)
# L /= np.linalg.norm(leadfield, axis=0)
alpha = 0.00 * np.linalg.svd(L)[1].max()

I = np.identity(n_chans)
C_inv = np.linalg.inv(y@y.T + alpha * I)
W = C_inv @ L @ np.linalg.inv(L.T @ C_inv @ L)
# W = W / np.linalg.norm(W, axis=0)

inverse_operator = W.T
x_hat = inverse_operator@y
stc_ = stc.copy()
stc_.data = x_hat / x_hat.max()
stc_.plot(**pp, brain_kwargs=dict(title="MCMV"))

Using control points [0.01444086 0.01927834 0.68025318]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Multiple Constrained Eigenspace MV Beamformer

In [11]:
from invert.util import find_corner
alpha = 0.00 * np.linalg.svd(leadfield)[1].max()

I = np.identity(n_chans)
C = y@y.T

U, s, _ = np.linalg.svd(C)
j = find_corner(np.arange(len(s)), s)

Us = U[:, :j]
Un = U[:, j:]


C_inv = np.linalg.inv(C + alpha * I)

W = []
for i in range(n_dipoles):
    l = leadfield[:, i][:, np.newaxis]
    # w = np.linalg.inv(l.T @ C_inv @ l ) * l.T @ C_inv
    w_mv = C_inv @ l *(1 / (l.T @ C_inv @ l))
    w_esmv = Us @ Us.T @ w_mv
    W.append(w_esmv)

W = np.stack(W, axis=1)[:, :, 0]
# W = W / np.linalg.norm(W, axis=0)

inverse_operator = W.T
x_hat = inverse_operator@y
stc_ = stc.copy()
stc_.data = x_hat / x_hat.max()
stc_.plot(**pp, brain_kwargs=dict(title="EIG-MCMV"))

Using control points [4.24658099e-15 8.17663745e-15 4.87323889e-01]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# ReciPSIICOS Projection

In [65]:
from invert.util import find_corner

G = deepcopy(leadfield)
G -= G.mean(axis=0)
# G_pwr = np.stack([g[:, np.newaxis] @ g[:, np.newaxis].T for g in G], axis=0)

# Prepare Data Covariance Matrix
I = np.identity(n_chans)
lam = 0.00
alpha = np.linalg.svd(y@y.T)[1].max()
C = y@y.T + lam * alpha * I

# Step 1
G_pwr = np.stack([g * g for g in G], axis=0)

# Step 2
U_pwr, S_pwr, V_pwr = np.linalg.svd(G_pwr, full_matrices=False)
k = find_corner(np.arange(len(S_pwr)), S_pwr)
P = U_pwr[:, :k] @ U_pwr[:, :k].T

# Step 3
C_x = (P@C).T #np.stack([p*c for p, c in zip(P, C)], axis=0).T

# Step 4
E, A, _ = np.linalg.svd(C_x, full_matrices=False)
C_x = E @ np.diag(np.abs(A)) @ E.T
C_x_inv = np.linalg.inv(C_x)

W = []
for i in range(n_dipoles):
    l = leadfield[:, i][:, np.newaxis]
    w = np.linalg.inv(l.T @ C_x_inv @ l) @ l.T @ C_x_inv
    W.append(w)
W = np.stack(W, axis=1)[0].T

# W /= np.linalg.norm(W, axis=0)
x_hat = W.T @ y

stc_ = stc.copy()
stc_.data = x_hat / x_hat.max()
stc_.plot(**pp, brain_kwargs=dict(title="ReciPSIICOS"))

Using control points [3.47266504e-13 5.81752306e-13 5.15981522e-01]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [2.49137208e-08 3.02073373e-08 5.01707312e-01]
Using control points [0.00000000e+00 0.00000000e+00 1.48756128e-08]
Using control points [3.13841874e-13 4.98840949e-13 3.16580251e-01]
Using control points [2.49137208e-08 3.02073373e-08 5.01707312e-01]
Using control points [3.13841874e-13 4.98840949e-13 3.16580251e-01]
Using control points [0.00000000e+00 0.00000000e+00 1.48756128e-08]
